In [46]:
%cd /home/jupyter

# Fix numpy and pandas versions (CRITICAL!)
# !pip install "numpy<2" "pandas==2.2.2" 

# Install required packages
!pip install --upgrade --quiet google-cloud-aiplatform
!pip install feast dvc[gs] scikit-learn 

print("✅ All packages installed successfully!")
!feast --version


/home/jupyter


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
matplotlib 3.7.3 requires numpy<2,>=1.20, but you have numpy 2.2.6 which is incompatible.
numba 0.58.1 requires numpy<1.27,>=1.22, but you have numpy 2.2.6 which is incompatible.
ydata-profiling 4.6.0 requires numpy<1.26,>=1.16.0, but you have numpy 2.2.6 which is incompatible.
ydata-profiling 4.6.0 requires pandas!=1.4.0,<2.1,>1.1, but you have pandas 2.2.2 which is incompatible.
ydata-profiling 4.6.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.10.6 which is inco

In [47]:
PROJECT_ID = "uplifted-earth-468613-f3"
BUCKET_NAME = "heart-disease-mlops-data"
LOCATION = "us-central1"

!gcloud config set project {PROJECT_ID}
print(f"✅ Project: {PROJECT_ID}")


Updated property [core/project].
✅ Project: uplifted-earth-468613-f3


In [4]:
!dvc get https://github.com/mudgalma/Real_World_Mlops data/heart.csv
print("✅ Downloaded heart.csv from DVC remote storage")


ERROR: failed to get 'data/heart.csv' from 'https://github.com/mudgalma/Real_World_Mlops' - The file 'heart.csv' already exists locally. 
To override it, re-run with '--force'.
✅ Downloaded heart.csv from DVC remote storage


In [32]:
import pandas as pd

df = pd.read_csv('dataset.csv')
print(f"✅ Loaded {len(df)} records")
print(f"Columns: {list(df.columns)}")
print("\nFirst 5 rows:")
print(df.head())
print(f"\nData shape: {df.shape}")


✅ Loaded 303 records
Columns: ['sno', 'age', 'gender', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

First 5 rows:
   sno  age  gender  cp  trestbps   chol  fbs  restecg  thalach  exang  \
0    0   63    male   3     145.0  233.0    1        0    150.0      0   
1    1   37    male   2     130.0  250.0    0        1    187.0      0   
2    2   41  female   1     130.0  204.0    0        0    172.0      0   
3    3   56    male   1     120.0  236.0    0        1    178.0      0   
4    4   57  female   0       NaN  354.0    0        1    163.0      1   

   oldpeak  slope  ca  thal target  
0      2.3      0   0     1    yes  
1      3.5      0   0     2    yes  
2      1.4      2   0     2    yes  
3      0.8      2   0     2    yes  
4      0.6      2   0     2    yes  

Data shape: (303, 15)


In [48]:
!pip install "numpy<2" "pandas==2.2.2"

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
feast 0.56.0 requires numpy<3,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.0 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.0 requires pandas!=1.4.0,<2.1,>1.1, but you have pandas 2.2.2 which is incompatible.
ydata-profiling 4.6.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.10.6 which is incompatible.
ydata-profiling 4.6.0 requires scipy<1.12,>=1.4.1, but you have scipy 1.15.3 whic

In [49]:
from datetime import datetime
import os
import pandas as pd

# Add required columns for Feast
df['patient_id'] = range(len(df))
df['timestamp'] = pd.to_datetime(datetime.utcnow())

# Create directory
os.makedirs('feast_data', exist_ok=True)

# Try Parquet first, fallback to CSV
try:
    import pyarrow
    df.to_parquet('feast_data/heart_features.parquet', index=False)
    print("✅ Saved to Parquet for Feast")
    FILE_FORMAT = "parquet"
except ImportError:
    df.to_csv('feast_data/heart_features.csv', index=False)
    print("⚠️ Saved to CSV (Parquet unavailable)")
    FILE_FORMAT = "csv"

print(f"File format: {FILE_FORMAT}")


✅ Saved to Parquet for Feast
File format: parquet


In [50]:
if FILE_FORMAT == "parquet":
    !gsutil cp feast_data/heart_features.parquet gs://{BUCKET_NAME}/feast/data/
    print(f"✅ Uploaded Parquet to gs://{BUCKET_NAME}/feast/data/")
else:
    !gsutil cp feast_data/heart_features.csv gs://{BUCKET_NAME}/feast/data/
    print(f"✅ Uploaded CSV to gs://{BUCKET_NAME}/feast/data/")


Copying file://feast_data/heart_features.parquet [Content-Type=application/octet-stream]...
- [1 files][ 16.0 KiB/ 16.0 KiB]                                                
Operation completed over 1 objects/16.0 KiB.                                     
✅ Uploaded Parquet to gs://heart-disease-mlops-data/feast/data/


In [51]:
!feast init heartproject
print("✅ Feast project initialized")
!ls -la heartproject/feature_repo/



Creating a new Feast repository in /home/jupyter/heartproject.

✅ Feast project initialized
total 40
drwxr-xr-x 5 jupyter jupyter 4096 Nov  9 09:38 .
drwxr-xr-x 4 jupyter jupyter 4096 Nov  9 12:43 ..
drwxr-xr-x 2 jupyter jupyter 4096 Nov  9 08:57 .ipynb_checkpoints
-rw-r--r-- 1 jupyter jupyter    0 Nov  9 09:38 __init__.py
drwxr-xr-x 2 jupyter jupyter 4096 Nov  9 09:38 __pycache__
drwxr-xr-x 2 jupyter jupyter 4096 Nov  9 06:05 data
-rw-r--r-- 1 jupyter jupyter 5249 Nov  9 12:43 example_repo.py
-rw-r--r-- 1 jupyter jupyter  537 Nov  9 12:43 feature_store.yaml
-rw-r--r-- 1 jupyter jupyter 4378 Nov  9 09:38 test_workflow.py


In [52]:
# Install Redis
!sudo apt-get update -qq
!sudo apt-get install -y redis-server -qq

# Start Redis in background
!redis-server --daemonize yes

# Verify Redis is running
!redis-cli ping

print("✅ Redis online store ready!")


PONG
✅ Redis online store ready!


In [39]:
%%writefile heartproject/feature_repo/feature_store.yaml
project: heart_disease_mlops
registry: gs://heart-disease-mlops-data/feast/registry.db
provider: local
offline_store:
  type: file
online_store:
  type: redis
  connection_string: "localhost:6379"
entity_key_serialization_version: 2



Overwriting heartproject/feature_repo/feature_store.yaml


In [56]:
%%writefile heartproject/feature_repo/example_repo.py
from feast import Entity, FeatureView, Field, FileSource
from feast.types import Float32, Int64
from datetime import timedelta

# Define patient entity
patient_entity = Entity(name="patient_id", join_keys=["patient_id"])

# Define data source (auto-detects Parquet or CSV)
heart_source = FileSource(
    path="../../feast_data/heart_features.parquet",
    timestamp_field="timestamp",
)

# Define feature view
heart_features = FeatureView(
    name="heart_features",
    entities=[patient_entity],
    ttl=timedelta(days=365),
    schema=[
        Field(name="age", dtype=Int64),
        Field(name="sex", dtype=Int64),
        Field(name="cp", dtype=Int64),
        Field(name="trestbps", dtype=Int64),
        Field(name="chol", dtype=Int64),
        Field(name="fbs", dtype=Int64),
        Field(name="restecg", dtype=Int64),
        Field(name="thalach", dtype=Int64),
        Field(name="exang", dtype=Int64),
        Field(name="oldpeak", dtype=Float32),
        Field(name="slope", dtype=Int64),
        Field(name="ca", dtype=Int64),
        Field(name="thal", dtype=Int64),
    ],
    online=True,
    source=heart_source,
    tags={"source": "dvc_github", "project": "heart_disease", "bucket": "gcs"},
)


Overwriting heartproject/feature_repo/example_repo.py


In [57]:
!cd heartproject/feature_repo && feast apply

print(f"\n✅ Feast registry saved to gs://{BUCKET_NAME}/feast/registry.db")
print("\n📋 Check GCS registry:")
!gsutil ls -l gs://{BUCKET_NAME}/feast/


Traceback (most recent call last):
  File "/home/jupyter/.local/lib/python3.10/site-packages/feast/importer.py", line 26, in import_class
    module = importlib.import_module(module_name)
  File "/opt/conda/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/jupyter/.local/lib/python3.10/site-packages/feast/infra/online_stores/sqlite.py", line 17, in <module>
    import sqlite3
  File "/opt/conda/lib/python3.10/sqlite3/__init__.py", line 57, in <module>
    from sqlite3

In [55]:
from feast import FeatureStore

store = FeatureStore(repo_path="heartproject/feature_repo")

print("📋 Registered Feature Views:")
for fv in store.list_feature_views():
    print(f"  - {fv.name}")
    
print("\n📋 Registered Entities:")
for entity in store.list_entities():
    print(f"  - {entity.name}")


📋 Registered Feature Views:

📋 Registered Entities:


In [14]:
from datetime import datetime, timedelta
import pandas as pd

# Load data to get actual timestamp range (like stock example)
if FILE_FORMAT == "parquet":
    df_check = pd.read_parquet('feast_data/heart_features.parquet')
else:
    df_check = pd.read_csv('feast_data/heart_features.csv')

df_check['timestamp'] = pd.to_datetime(df_check['timestamp'])

start_time = df_check['timestamp'].min()
end_time = df_check['timestamp'].max()

print(f"Data timestamp range: {start_time} to {end_time}")

# Materialize with actual data range
store.materialize(start_time, end_time)
print(f"\n✅ Features materialized from {start_time} to {end_time}")


Data timestamp range: 2025-11-09 12:18:06.161873 to 2025-11-09 12:18:06.161873
Materializing 0 feature views from 2025-11-09 12:18:06+00:00 to 2025-11-09 12:18:06+00:00 into the sqlite online store.


✅ Features materialized from 2025-11-09 12:18:06.161873 to 2025-11-09 12:18:06.161873


In [ ]:
# Get online features for real-time prediction
online_features = store.get_online_features(
    features=[
        "heart_features:age",
        "heart_features:sex",
        "heart_features:chol",
        "heart_features:trestbps",
        "heart_features:thalach",
    ],
    entity_rows=[
        {"patient_id": 0},
        {"patient_id": 1},
        {"patient_id": 2},
    ],
).to_df()

print("✅ Online features retrieved (for real-time prediction):")
print(online_features)


In [ ]:
# Create entity dataframe for training
entity_df = pd.DataFrame({
    "patient_id": range(20),
    "timestamp": [datetime.utcnow()] * 20
})

# Get historical features
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "heart_features:age",
        "heart_features:sex",
        "heart_features:cp",
        "heart_features:chol",
        "heart_features:trestbps",
        "heart_features:thalach",
        "heart_features:oldpeak",
    ],
).to_df()

print("✅ Historical features for model training:")
print(training_df.head(10))
print(f"\nShape: {training_df.shape}")


In [ ]:
import json

print("\n" + "="*70)
print("FEAST + DVC INTEGRATION - COMPLETED SUCCESSFULLY ✅")
print("="*70)

print(f"\n📊 Data Source:")
print(f"  - DVC Repository: https://github.com/mudgalma/Real_World_Mlops")
print(f"  - File: data/heart.csv")
print(f"  - Records: {len(df)}")
print(f"  - Format: {FILE_FORMAT.upper()}")

print(f"\n🗄️ Feast Configuration:")
print(f"  - Registry: gs://{BUCKET_NAME}/feast/registry.db")
print(f"  - Offline Store: File (CSV/Parquet)")
print(f"  - Online Store: SQLite (data/online_store.db)")
print(f"  - Features: 13 clinical measurements")

print(f"\n✅ Operations Completed:")
print(f"  1. Downloaded data from DVC-tracked repository")
print(f"  2. Prepared data with patient_id and timestamps")
print(f"  3. Uploaded to GCS bucket")
print(f"  4. Registered feature definitions with Feast")
print(f"  5. Materialized features to online store (SQLite)")
print(f"  6. Retrieved online features (real-time serving)")
print(f"  7. Retrieved historical features (model training)")

print(f"\n📁 Files Created:")
print(f"  - Local: feast_data/heart_features.{FILE_FORMAT}")
print(f"  - GCS: gs://{BUCKET_NAME}/feast/data/heart_features.{FILE_FORMAT}")
print(f"  - GCS: gs://{BUCKET_NAME}/feast/registry.db")
print(f"  - Local: heartproject/feature_repo/data/online_store.db")

print(f"\n📝 Next Steps:")
print(f"  1. Download this notebook (.ipynb)")
print(f"  2. Upload to GitHub: notebooks/feast_heart_disease_demo.ipynb")
print(f"  3. Update main README with Feast integration section")
print(f"  4. (Optional) Deploy feature server for production serving")

print("\n" + "="*70)
